In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
%load_ext tensorboard
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32
#constant
BATCH_SIZE = 128
EPOCHS = 20
CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = tf.keras.optimizers.RMSprop()

In [2]:
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
 
#normalize 
mean = np.mean(X_train,axis=(0,1,2,3))
std = np.std(X_train,axis=(0,1,2,3))
X_train = (X_train-mean)/(std+1e-7)
X_test = (X_test-mean)/(std+1e-7)

y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

In [3]:
#image augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )
datagen.fit(X_train)

In [4]:
def build(): 
    model = models.Sequential()
    
    #1st block
    model.add(layers.Conv2D(32, (3,3), padding='same', 
        input_shape=X_train.shape[1:], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2))
    #2nd block
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))
    #3d block 
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4))
    #dense  
    model.add(layers.Flatten())
    model.add(layers.Dense(CLASSES, activation='softmax'))
    return model
    

In [5]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./logs/f')]

In [6]:
model = build()

In [7]:
model.compile(loss='categorical_crossentropy',optimizer=OPTIM,
              metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=BATCH_SIZE,
          epochs=EPOCHS, validation_split=VALIDATION_SPLIT,
          verbose= VERBOSE,callbacks=callbacks)
score=model.evaluate(X_test,y_test,
                     batch_size=BATCH_SIZE,verbose= VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

Epoch 1/20
313/313 [==============================] - 17s 14ms/step - loss: 1.9529 - accuracy: 0.4227 - val_loss: 1.3852 - val_accuracy: 0.5220
Epoch 2/20
313/313 [==============================] - 4s 12ms/step - loss: 1.2562 - accuracy: 0.5958 - val_loss: 1.1300 - val_accuracy: 0.6354
Epoch 3/20
313/313 [==============================] - 4s 12ms/step - loss: 1.0144 - accuracy: 0.6687 - val_loss: 1.0036 - val_accuracy: 0.6745
Epoch 4/20
313/313 [==============================] - 4s 12ms/step - loss: 0.8773 - accuracy: 0.7130 - val_loss: 0.7968 - val_accuracy: 0.7323
Epoch 5/20
313/313 [==============================] - 4s 12ms/step - loss: 0.7686 - accuracy: 0.7416 - val_loss: 0.7296 - val_accuracy: 0.7593
Epoch 6/20
313/313 [==============================] - 4s 11ms/step - loss: 0.6917 - accuracy: 0.7673 - val_loss: 0.6889 - val_accuracy: 0.7654
Epoch 7/20
313/313 [==============================] - 4s 12ms/step - loss: 0.6400 - accuracy: 0.7856 - val_loss: 0.6916 - val_accuracy: 0.772

In [9]:
%tensorboard --logdir logs/f

In [10]:
model_json = model.to_json()
with open('cifar10_architecture.json', 'w') as json_file:
    json_file.write(model_json)

In [11]:
model.save_weights('model.h5')

In [12]:
scores = model.evaluate(X_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0])) 

79/79 [==============================] - 1s 7ms/step - loss: 0.5780 - accuracy: 0.8244

Test result: 82.440 loss: 0.578
